**Imports**

In [1]:
import numpy as np
import random as rand
from sklearn.model_selection import KFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.linear_model import SGDClassifier
import math as ma
import scipy as sp
import matplotlib.pyplot as plt
import pandas as pd
import time
print("Finished importing!")

Finished importing!


**Mounting Google Drive**


In [2]:
# from google.colab import drive
# drive.mount('/content/myDrive')

**Restrict the Number of Training Samples for Testing Purposes**

In [3]:
# Removes samples from the training set until only remaining_samples samples remain.
def sampleReduction(training_set,remaining_samples):
  count = len(training_set) - remaining_samples

  while count > 0:
    training_set.pop()
    count -= 1

  return training_set

**Importing Data Sets**

In [4]:
# df_train = pd.read_csv('/content/myDrive/My Drive/ECSE_551_Machine_Learning/train.csv')
# df_test = pd.read_csv('/content/myDrive/My Drive/ECSE_551_Machine_Learning/test.csv')
# df_train = pd.read_csv('C:/Users/AlexG35/Desktop/GitHub/TextClassification/train.csv')
# df_test = pd.read_csv('C:/Users/AlexG35/Desktop/GitHub/TextClassification/test.csv')
# df_train = pd.read_csv('/Users/jhu69/Desktop/Github/TextClassification/train.csv')
# df_test = pd.read_csv('/Users/jhu69/Desktop/Github/TextClassification/test.csv')
df_train = pd.read_csv('C:/Users/karan/Desktop/Masters/Codes/CodeRemote/GitHub_codes/Under_Grad_mSpice/DP_teams/TextClassification/train.csv')
df_test = pd.read_csv('C:/Users/karan/Desktop/Masters/Codes/CodeRemote/GitHub_codes/Under_Grad_mSpice/DP_teams/TextClassification/test.csv')

# Converting DataFrames to lists.
training_set = df_train.values.tolist()
test_set = df_test.values.tolist()

# Shuffling the training and test data sets
rand.shuffle(training_set)
rand.shuffle(test_set)

#--------Restrict the Number of Training Samples for Testing Purposes**---------
print("Original number of training samples: " + str(len(training_set)))
training_set = sampleReduction(training_set, 2000)
print("Reduced number of training samples: " + str(len(training_set)))
#--------Restrict the Number of Training Samples for Testing Purposes**---------

# print(training_set[0])

# Splitting the sets into inputs and outputs.
training_set_in = []
training_set_out = []
test_set_in = []
# Note: no outputs for test set - must predict them.

for sample in training_set:
  training_set_in.append(sample[0])
  training_set_out.append(sample[1])

for sample in test_set:
  test_set_in.append(sample[1])

# print(training_set_out)
  

Original number of training samples: 11582
Reduced number of training samples: 2000


**Parsing**


In [5]:
# Dictionary of relevant characters in a word.
# Returns true if the character is a *LOWERCASE* letter, number or accepted 
# special character, else returns false.
def isAlphanumeric (char):
  switcher = {
    'a': True,
    'b': True,
    'c': True,
    'd': True,
    'e': True,
    'f': True,
    'g': True,
    'h': True,
    'i': True,
    'j': True,
    'k': True,
    'l': True,
    'm': True,
    'n': True,
    'o': True,
    'p': True,
    'q': True,
    'r': True,
    's': True,
    't': True,
    'u': True,
    'v': True,
    'w': True,
    'x': True,
    'y': True,
    'z': True,
    '0': True,
    '1': True,
    '2': True,
    '3': True,
    '4': True,
    '5': True,
    '6': True,
    '7': True,
    '8': True,
    '9': True,
    '\'': True,   #Apostrophe
    '$': True,
    #'@': True,    #For email addresses
    #'_': True, 
    '-': True,
    #'/': True     #For websites
  }
  return switcher.get(char,False)

In [6]:
# Returns true if character is a number, false otherwise.
def isNumber(char):
  switcher = {
    '0': True,
    '1': True,
    '2': True,
    '3': True,
    '4': True,
    '5': True,
    '6': True,
    '7': True,
    '8': True,
    '9': True,
  }
  return switcher.get(char,False)

In [7]:
# Joins the characters in a list of characters together to form a string.
def mergeCharacters(char_array):
  # Intialize string to ""
  s = ""

  return(s.join(char_array))

In [8]:
# Parses the input string and returns its words in the form of a list of strings.
def parseLine(line):
  line = line.lower()     # Converts all letters to lowercase
  char_array = list(line) # Converts the line to an list of characters
  words_array = [[]]      # List of a list of characters that for words
  words = []              # List of words
  word_idx = 0      # Index of the current word

  # Creates a list of words made out of characters (i.e. a list of lists of char).
  for char in char_array:
    if isAlphanumeric(char):
      temp = words_array[word_idx]    # temp is the current word
      temp.append(char)
      #char_idx += 1
    else:
      if words_array[word_idx]:     # True if word contains at least 1 character
        words_array.append([])      # Appending the next empty word to be filled
        word_idx += 1               # Index into the next word

  # Creates a list of words made out of strings.
  for word in words_array:
    if word:                        # If word is non-empty. Always true (hopefully) except for the last one)            
      dollar_sign = False
      contains_number = False
      for char in word:
        if char == '$':                
          dollar_sign = True
        elif isNumber(char):
          contains_number = True
      if dollar_sign:
        words.append("money")       # Substitute any money amount with simply "money"
      elif contains_number:
        words.append("number")      # Substitute any number with simply "number"
      else:
        words.append(mergeCharacters(word))  # Merge the characters of the word into a string

  return words

In [9]:
# Converts the inputs of the training and test sets into lists of words in the form of strings.
X_words = []
Xtest_words = []

for sample in training_set_in:
  X_words.append(parseLine(sample))

# NO NEED TO PARSE THE TEST SET FOR NOW
# for sample in test_set_in:
#  Xtest_words.append(parseLine(sample))
print("Converted training and test sets into a list of lists of words!")
print("Each row in X_words corresponds to a training sample represented as a list of words")


Converted training and test sets into a list of lists of words!
Each row in X_words corresponds to a training sample represented as a list of words


**Feature Removal and Visualization**

In [10]:
#@title Dictionary of Irrelevant Words (need to run this)
# Dictionary of irrelevant words.
# Returns true if the *LOWERCASE* word is irrelevant, else returns false.
def isIrrelevant (word):
  switcher = {
    # All pronouns and associated words
    "i": True,
    "i'll": True,
    "i'd": True,
    "i'm": True,
    "i've": True,
    "ive": True,
    "me": True,
    "myself": True,
    "you": True,
    "you'll": True,
    "you'd": True,
    "you're": True,
    "you've": True,
    "yourself": True,
    "he": True,
    "he'll": True,
    "he'd": True,
    "he's": True,
    "him": True,
    "she": True,
    "she'll": True,
    "she'd": True,
    "she's": True,
    "her": True,
    "it": True,
    "it'll": True,
    "it'd": True,
    "it's": True,
    "itself": True,
    "oneself": True,
    "we": True,
    "we'll": True,
    "we'd": True,
    "we're": True,
    "we've": True,
    "us": True,
    "ourselves": True,
    "they": True,
    "they'll": True,
    "they'd": True,
    "they're": True,
    "they've": True,
    "them": True,
    "themselves": True,            
    "everyone": True,
    "everyone's": True,
    "everybody": True,
    "everybody's": True,
    "someone": True,
    "someone's": True,
    "somebody": True,
    "somebody's": True,
    "nobody": True,
    "nobody's": True,
    "anyone": True,
    "anyone's": True,
    "everything": True,
    "everything's": True,
    "something": True,
    "something's": True,
    "nothing": True,
    "nothing's": True,
    "anything": True,
    "anything's": True,
    # All determiners and associated words
    "a": True,
    "an": True,
    "the": True,
    "this": True,
    "that": True,
    "that's": True,
    "these": True,
    "those": True,
    "my": True,
    #"mine": True,   #Omitted since mine can refer to something else
    "your": True,
    "yours": True,
    "his": True,
    "hers": True,
    "its": True,
    "our": True,
    "ours": True,
    "own": True,
    "their": True,
    "theirs": True,
    "few": True,
    "much": True,
    "many": True,
    "lot": True,
    "lots": True,
    "some": True,
    "any": True,
    "enough": True,
    "all": True,
    "both": True,
    "half": True,
    "either": True,
    "neither": True,
    "each": True,
    "every": True,
    "certain": True,
    "other": True,
    "another": True,
    "such": True,
    "several": True,
    "multiple": True,
    # "what": True,    #Dealt with later on
    "rather": True,
    "quite": True,
    # All prepositions
    "aboard": True,
    "about": True,
    "above": True,
    "across": True,
    "after": True,
    "against": True,
    "along": True,
    "amid": True,
    "amidst": True,
    "among": True,
    "amongst": True,
    "anti": True,
    "around": True,
    "as": True,
    "at": True,
    "away": True,
    "before": True,
    "behind": True,
    "below": True,
    "beneath": True,
    "beside": True,
    "besides": True,
    "between": True,
    "beyond": True,
    "but": True,
    "by": True,
    "concerning": True,
    "considering": True,
    "despite": True,
    "down": True,
    "during": True,
    "except": True,
    "excepting": True,
    "excluding": True,
    "far": True,
    "following": True,
    "for": True,
    "from": True,
    "here": True,
    "here's": True,
    "in": True,
    "inside": True,
    "into": True,
    "left": True,
    "like": True,
    "minus": True,
    "near": True,
    "of": True,
    "off": True,
    "on": True,
    "onto": True,
    "opposite": True,
    "out": True,
    "outside": True,
    "over": True,
    "past": True,
    "per": True,
    "plus": True,
    "regarding": True,
    "right": True,
    #"round": True,   #Omitted
    #"save": True,    #Omitted
    "since": True,
    "than": True,
    "there": True,
    "there's": True,
    "through": True,
    "to": True,
    "toward": True,
    "towards": True,
    "under": True,
    "underneath": True,
    "unlike": True,
    "until": True,
    "up": True,
    "upon": True,
    "versus": True,
    "via": True,
    "with": True,
    "within": True,
    "without": True,
    # Irrelevant verbs
    "may": True,
    "might": True,
    "will": True,
    "won't": True,
    "would": True,
    "wouldn't": True,
    "can": True,
    "can't": True,
    "cannot": True,
    "could": True,
    "couldn't": True,
    "should": True,
    "shouldn't": True,
    "must": True,
    "must've": True,
    "be": True,
    "being": True,
    "been": True,
    "am": True,
    "are": True,
    "aren't": True,
    "ain't": True,
    "is": True,
    "isn't": True,
    "was": True,
    "wasn't": True,
    "were": True,
    "weren't": True,
    "do": True,
    "doing": True,
    "don't": True,
    "does": True,
    "doesn't": True,
    "did": True,
    "didn't": True,
    "done": True,
    "have": True,
    "haven't": True,
    "having": True,
    "has": True,
    "hasn't": True,
    "had": True,
    "hadn't": True,
    "get": True,
    "getting": True,
    "gets": True,
    "got": True,
    "gotten": True,
    "go": True,
    "going": True,
    "gonna": True,
    "goes": True,
    "went": True,
    "gone": True,
    "make": True,
    "making": True,
    "makes": True,
    "made": True,
    "take": True,
    "taking": True,
    "takes": True,
    "took": True,
    "taken": True,
    "need": True,
    "needing": True,
    "needs": True,
    "needed": True,
    "use": True,
    "using": True,
    "uses": True,
    "used": True,
    "want": True,
    "wanna": True,
    "wanting": True,
    "wants": True,
    "let": True,
    "lets": True,
    "letting": True,
    "let's": True,
    "suppose": True,
    "supposing": True,
    "supposes": True,
    "supposed": True,
    "seem": True,
    "seeming": True,
    "seems": True,
    "seemed": True,
    "say": True,
    "saying": True,
    "says": True,
    "said": True,
    "know": True,
    "knowing": True,
    "knows": True,
    "knew": True,
    "known": True,
    "look": True,
    "looking": True,
    "looked": True,
    "think": True,
    "thinking": True,
    "thinks": True,
    "thought": True,
    "feel": True,
    "feels": True,
    "felt": True,
    "based": True,
    "put": True,
    "puts": True,
    #"wanted": True   #Omitted since the advective is relevant
    # Question words and associated words
    "who": True,
    "who's": True,
    "who've": True,
    "who'd": True,
    "whoever": True,
    "whoever's": True,
    "whom": True,
    "whomever": True,
    "whomever's": True,
    "whose": True,
    "whosever": True,
    "whosever's": True,
    "when": True,
    "whenever": True,
    "which": True,
    "whichever": True,
    "where": True,
    "where's": True,
    "where'd": True,
    "wherever": True,
    "why": True,
    "why's": True,
    "why'd": True,
    "whyever": True,
    "what": True,
    "what's": True,
    "whatever": True,
    "whence": True,
    "how": True,
    "how's": True,
    "how'd": True,
    "however": True,
    "whether": True,
    "whatsoever": True,
    # Connector words and irrelevant adverbs
    "and": True,
    "or": True,
    "not": True,
    "because": True,
    "also": True,
    "always": True,
    "never": True,
    "only": True,
    "really": True,
    "very": True,
    "greatly": True,
    "extremely": True,
    "somewhat": True,
    "no": True,
    "nope": True,
    "nah": True,
    "yes": True,
    "yep": True,
    "yeh": True,
    "yeah": True,
    "maybe": True,
    "perhaps": True,
    "more": True,
    "most": True,
    "less": True,
    "least": True,
    "good": True,
    "great": True,
    "well": True,
    "better": True,
    "best": True,
    "bad": True,
    "worse": True,
    "worst": True,
    "too": True,
    "thru": True,
    "though": True,
    "although": True,
    "yet": True,
    "already": True,
    "then": True,
    "even": True,
    "now": True,
    "sometimes": True,
    "still": True,
    "together": True,
    "altogether": True,
    "entirely": True,
    "fully": True,
    "entire": True,
    "whole": True,
    "completely": True,
    "utterly": True,
    "seemingly": True,
    "apparently": True,
    "clearly": True,
    "obviously": True,
    "actually": True,
    "actual": True,
    "usually": True,
    "usual": True,
    "literally": True,
    "honestly": True,
    "absolutely": True,
    "definitely": True,
    "generally": True,
    "totally": True,
    "finally": True,
    "basically": True,
    "essentially": True,
    "fundamentally": True,
    "automatically": True,
    "immediately": True,
    "necessarily": True,
    "primarily": True,
    "normally": True,
    "perfectly": True,
    "constantly": True,
    "particularly": True,
    "eventually": True,
    "hopefully": True,
    "mainly": True,
    "typically": True,
    "specifically": True,
    "differently": True,
    "appropriately": True,
    "plenty": True,
    "certainly": True,
    "unfortunately": True,
    "ultimately": True,
    "unlikely": True,
    "likely": True,
    "potentially": True,
    "fortunately": True,
    "personally": True,
    "directly": True,
    "indirectly": True,
    "nearly": True,
    "closely": True,
    "slightly": True,
    "probably": True,
    "possibly": True,
    "especially": True,
    "frequently": True,
    "often": True,
    "oftentimes": True,
    "seldom": True,
    "rarely": True,
    "sure": True,
    "while": True,
    "whilst": True,
    "able": True,
    "unable": True,
    "else": True,
    "ever": True,
    "once": True,
    "twice": True,
    "thrice": True,
    "almost": True,
    "again": True,
    "instead": True,
    "next": True,
    "previous": True,
    "unless": True,
    "somehow": True,
    "anyhow": True,
    "anywhere": True,
    "somewhere": True,
    "everywhere": True,
    "nowhere": True,
    "further": True,
    "anymore": True,
    "later": True,
    "ago": True,
    "ahead": True,
    "just": True,
    "same": True,
    "different": True,
    "big": True,
    "small": True,
    "little": True,
    "tiny": True,
    "large": True,
    "huge": True,
    "pretty": True,
    "mostly": True,
    "anyway": True,
    "anyways": True,
    "otherwise": True,
    "regardless": True,
    "throughout": True,
    "additionally": True,
    "moreover": True,
    "furthermore": True,
    "meanwhile": True,
    "afterwards": True,
    # Irrelevant nouns
    "thing": True,
    "thing's": True,
    "things": True,
    "stuff": True,
    "other's": True,
    "others": True,
    "another's": True,
    "total": True,
    "true": True,
    "false": True,
    "none": True,
    "way": True,
    "kind": True,
    # Lettered numbers and order
    "zero": True,
    "zeros": True,
    "zeroes": True,
    "one": True,
    "ones": True,
    "two": True,
    "three": True,
    "four": True,
    "five": True,
    "six": True, 
    "seven": True,
    "eight": True,
    "nine": True,
    "ten": True,
    "twenty": True,
    "thirty": True,
    "forty": True,
    "fifty": True,
    "sixty": True,
    "seventy": True,
    "eighty": True,
    "ninety": True,
    "hundred": True,
    "hundreds": True,
    "thousand": True,
    "thousands": True,
    "million": True,
    "millions": True,
    "first": True,
    "last": True,
    "second": True,
    "third": True,
    "fourth": True,
    "fifth": True,
    "sixth": True,
    "seventh": True,
    "eigth": True,
    "ninth": True,
    "tenth": True,
    "firstly": True,
    "secondly": True,
    "thirdly": True,
    "lastly": True,
    # Greetings and slang
    "hello": True,
    "hi": True,
    "hey": True,
    "sup": True,
    "yo": True,
    "greetings": True,
    "please": True,
    "okay": True,
    "ok": True,
    "y'all": True,
    "lol": True,
    "rofl": True,
    "thank": True,
    "thanks": True,
    "alright": True,
    "kinda": True,
    "dont": True,
    "sorry": True,
    "idk": True,
    "tldr": True,
    "tl": True,
    "dr": True,  #This means that dr (doctor) is a bad feature because of tl;dr
    "tbh": True,
    "dude": True,
    "tho": True,
    "aka": True,
    "plz": True,
    "pls": True,
    "bit": True,
    "don": True,

    # Miscellaneous
    "www": True,
    "https": True,
    "http": True,
    "com": True,
    "etc": True,
    "html": True,
    "reddit": True,
    "subreddit": True,
    "subreddits": True,
    "comments": True,
    "reply": True,
    "replies": True,
    "thread": True,
    "threads": True,
    "post": True,
    "posts": True,
    "website": True,
    "websites": True,
    "web site": True,
    "web sites": True
  }
  return switcher.get(word,False)

In [11]:
# Bubble sort (descending order) for 2-dimensional list.
def bubbleSort(list):
  n = len(list) 
    
  for i in range(n): 
      swapped = False

      # Last i elements are already in place
      for j in range(0, n-i-1): 
  
          # loop through the array from 0 to n-i-1. Swap if
          # the element found is less than the next element
          if list[j][1] < list[j+1][1] : 
              list[j], list[j+1] = list[j+1], list[j] 
              swapped = True

      # If no two elements were swapped 
      # by the inner loop, then break 
      if swapped == False: 
          break

In [12]:
# TODO: Leminization follow the tutorial stuff and call it after the two for loops

# Remove all the irrelevant words from the entire X_words and Xtest_words
for i in range(0, len(X_words)): # i is the current sample
  j = 0
  while j < len(X_words[i]):
    if isIrrelevant(X_words[i][j]) or ((len(X_words[i][j]) < 3) and (X_words[i][j] != ("money" or "number"))):
      X_words[i].remove(X_words[i][j])  #Remove irrelevant words from the training set
      j -= 1
    j += 1

# NO NEED TO REMOVE THE IRRELEVANT WORDS IN Xtest FOR NOW
# for i in range(0, len(Xtest_words)): # i is the current sample
#   j = 0
#   while j < len(Xtest_words[i]):
#     if isIrrelevant(Xtest_words[i][j]) or ((len(Xtest_words[i][j]) < 3) and (Xtest_words[i][j] != ("money" or "number"))):
#       Xtest_words[i].remove(Xtest_words[i][j])  #Remove irrelevant words from the training set
#       j -= 1
#    j += 1

In [13]:
print(X_words[0])

['come', 'printable', 'quick', 'reference', 'sheet', 'foot', 'logh', 'leagues', 'adventure', 'share', 'googled', 'eyes', 'bleed', 'found', 'hex', "gm's", 'screen', 'char', 'gen', 'ubiquity', 'site', 'tia']


**Convert Selected Features into Numpy Array**

In [14]:
# Select the number of desired features (words) and then return a numpy array X
# with the top num_features most frequent features (words).
# num_features is essentially the desired dictionary size of useful words.
# The input disp_number is the number of top features to display.
def featureSelection(X_words, Xtest_words, num_features, disp_number):
    # Make a list of all the unique words in X_words and calculate their frequency.
    temp1 = []     # List of all the unique words
    temp2 = []     # List of all the words

    for i in range(0, len(X_words)):  # i is the current sample
        for j in range(0, len(X_words[i])):     # j is the current word number
            temp2.append(X_words[i][j])

            if X_words[i][j] not in temp1:    # Check for duplicates
                temp1.append(X_words[i][j])

    print(f"Length of ordered list: {len(temp1)}")

    # Make a list of each unique word present and its frequency.
    ordered_list = []
    for x in range(0, len(temp1)):
        ordered_list.append([temp1[x], temp2.count(temp1[x])])

    # Sort the words in descending order of frequency. NOTE: Computationally heavy!
    bubbleSort(ordered_list)

    # Display the frequency of each unique word present.
    # Note:  Max number of lines that can be displayed is 4999 in colab
    for y in range (0,disp_number):
        print('Frequency of', ordered_list[y][0], 'is:', ordered_list[y][1])

    # Take the first num_features features and convert X_words into X
    X = np.zeros((len(X_words),num_features))     # Numpy array of binary vectors
    Xtest = np.zeros((len(Xtest_words),num_features)) #Numpy array of binary vectors for val or test

    for i in range(0, X.shape[0]):         # i is the row of X_words and X
        for j in range(0, X.shape[1]):       # j is the feature (word) index
            if X_words[i].__contains__(ordered_list[j][0]):
                X[i,j] = 1               # Assign 1 if X_words[i] contains the word
            else:
                X[i,j] = 0               # Assign 0 if X_words[i] does not contain the word

    for i in range(0, Xtest.shape[0]):         # i is the row of Xtest_words and X
        for j in range(0, Xtest.shape[1]):       # j is the feature (word) index
            if Xtest_words[i].__contains__(ordered_list[j][0]):
                Xtest[i,j] = 1               # Assign 1 if Xtest_wrods[i] contains the word
            else:
                Xtest[i,j] = 0               # Assign 0 if Xtest_words[i] does not contain the word

    return X, Xtest

In [15]:
# Convert X_words into the numpy array X of zeroes and ones.
# Essentially, each sentence is now a binary vector of size num_features.  
# Note: do not set num_features in featureSelection too high. 5000 is the 
# minimum accepted number of features in the competition.
# X = featureSelection(X_words,5000,100)
# NOTE: K-Fold does not make use of the X calculated here, this is only for testing purposes
# print(f"The shape of X is: {X.shape}")

**Converting the Class Labels into a Numpy Array**

In [16]:
# Convert the class labels to integers from 0 to 7 using a dictionary
def labelToNumber (label):
  switcher = {
    "rpg": 0,
    "anime": 1,
    "datascience": 2,
    "hardware": 3,
    "cars": 4,
    "gamernews": 5,
    "gamedev": 6,
    "computers": 7,
  }
  return switcher.get(label,"Invalid class label")

In [17]:
# Convert the class labels into a numpy array named Y.
# Each entry in Y should still match the corresponding row in X.
Y = np.zeros((len(training_set_out),1))

for i in range(0, Y.shape[0]):
  # Convert the class labels to a number between 0 and 7
  label_number = labelToNumber(training_set_out[i])   

  if label_number != "Invalid class label":
    Y[i,0] = label_number
  else: 
    print("Invalid class label!")
      
print(f"The shape of our labels vector Y: {Y.shape}")

The shape of our labels vector Y: (2000, 1)


**Probabilities**

In [18]:
# Calculate the probability of Y = 1.
# Returns a column vector for which the ith entry is P(Yi=1)
# and i refers to a class label.
def probY(Y, num_classes):
  prob = np.zeros((num_classes,1))  #Probability vector P(Yi=1)
    
  for i in range(0,Y.shape[0]):
    # Assumption: labels are integers ranging from 0 to num_labels - 1
    for label in range(0, num_classes):
      if Y[i,0] == label:
        prob[label,0] += 1
      elif Y[i,0] > (num_classes - 1):
        print("Y at index " + str(i) + " is an invalid class label")
        break
  # Divide by the total # of labels input labels Y
  prob = prob/Y.shape[0]
  return prob

In [19]:
print(Y.shape)
probOfAllClassLabels = probY(Y,8)
print(probOfAllClassLabels.shape)
print(probOfAllClassLabels)

(2000, 1)
(8, 1)
[[0.174 ]
 [0.121 ]
 [0.2065]
 [0.1295]
 [0.184 ]
 [0.07  ]
 [0.0855]
 [0.0295]]


In [20]:
# Calculate the probability of X = 1 given a class label Yi (number).
# Returns a row vector for which the jth entry is P(Xj=1|Yi)
# and j refers to a feature.
# Note: not implemented with the bias term in mind.
def probXGivenYi(X, Y, label):
  prob = np.zeros((1,X.shape[1]))  # Conditional probability vector P(Xj=1|Yi)
  denominator = 0                  # Number of times label Yi appears in Y

  for i in range(0, X.shape[0]):
    if Y[i,0] == label:
      denominator += 1
      prob = prob + X[i,:]

  # Laplace smoothing
  prob = prob + np.ones((1,X.shape[1]))
  denominator += 2

  prob = prob/denominator
  return prob

In [21]:
# # Finding the conditional probability of features of X given we know its of a specific label
# probXgivenYi = probXGivenYi(X, Y, 1)
# print(f"Shape of P(Xj=1/Yi) is: {probXgivenYi.shape}")
# print(probXgivenYi)

Naive Bayes Classifier

In [22]:
# Superclass for Bernouilli Naive Bayes Classifiers
class Bernoulli_NB(object):


    #Class constructor
    def __init__(self, num_classes,LaplaceSmooth=True):
      self.num_classes = num_classes
      self.condProb = None
      self.priorProb = None
      self.LaplaceSmoothening = LaplaceSmooth
    ####################################################################################################
    ####################################################################################################

    # Calculate the probability of Y = 1.
    # Returns a column vector for which the ith entry is P(Yi=1)
    # and i refers to a class label.
    def probY(self, Y):
      num_labels = self.num_classes
      prob = np.zeros((num_labels,1))  #Probability vector P(Yi=1)

      for i in range(0,Y.shape[0]):
        # Assumption: labels are integers ranging from 0 to num_labels - 1
        for label in range(0, num_labels):
          if Y[i,0] == label:
            prob[label,0] += 1
          elif Y[i,0] > (num_labels - 1):
            print("Y at index " + str(i) + " is an invalid class label")
            break
      # Divide by the total # of labels input labels Y
      prob = prob/Y.shape[0]
      return prob

    ####################################################################################################
    ####################################################################################################
    # Calculate the probability of X = 1 given a class label Yi (number).
    # Returns a row vector for which the jth entry is P(Xj=1|Yi)
    # and j refers to a feature.
    # Note: not implemented with the bias term in mind.
    def probXGivenYi(self, X, Y, label):
      prob = np.zeros((1,X.shape[1]))  # Conditional probability vector P(Xj=1|Yi)
      denominator = 0                  # Number of times label Yi appears in Y

      for i in range(0, X.shape[0]):
        if Y[i,0] == label:
          denominator += 1
          prob = prob + X[i,:]

      # Laplace smoothing
      if(self.LaplaceSmoothening):
        prob = prob + np.ones((1,X.shape[1]))
        denominator += 2

      prob = prob/denominator
      return prob
    ####################################################################################################
    ####################################################################################################
    # Write the Bernoulli Naive  Bayes Method Here
    def fit(self, Xtrain, Y):
        print('Starting the fit function::::')
        t1 = time.time()

        self.condProb = np.zeros((self.num_classes,Xtrain.shape[1]))
        t2 =time.time()
        self.priorProb = self.probY(Y)
        print('Time taken for Pior probs function:::', time.time()-t2 )

        t3 = time.time()
        for c in range(self.num_classes):
            self.condProb[c,:] = self.probXGivenYi(Xtrain, Y, c)
        print('Time taken for Condtional Prob:::', time.time()-t3)
        print('Total time by fit function:::', time.time()-t1 )
    ####################################################################################################
    ####################################################################################################
                            #NON OPTIMISED BERNOULLI #
    # def predict(self,Xtest):
    #
    #     predLabel = np.zeros((Xtest.shape[0],1))
    #     prosteriorProb = np.zeros((Xtest.shape[0],8))
    #
    #     for d in range(Xtest.shape[0]):
    #         for c in range(8):
    #             prosteriorProb[d,c] = np.log10(self.priorProb[c,0])
    #             for index in range(Xtest.shape[1]):
    #                 if(Xtest[d,index] == 1):
    #                     prosteriorProb[d,c] += np.log10(self.condProb[c,index])
    #                 else:
    #                     prosteriorProb[d,c] += np.log10(1 - self.condProb[c,index])
    #
    #                 # print(self.condProb[c, index])
    #
    #         predLabel[d,0] = np.argmax(prosteriorProb[d,:])
    #         # print(prosteriorProb[d, :])
    #         # print(predLabel[d,0])
    #         # if d == 5:
    #         #     return
    #
    #     return predLabel
    ####################################################################################
    ####################################################################################
                            # OPTIMISED BERNOULLI #
    def predict(self,Xtest):
        print('Starting the Predicting Function: tick tock... tick tock')
        start_time = time.time()
        predLabel = np.zeros((Xtest.shape[0],1))
        prosteriorProb = np.zeros((Xtest.shape[0],self.num_classes))
        for d in range(Xtest.shape[0]):
            for c in range(self.num_classes):
                prosteriorProb[d,c] = np.log10(self.priorProb[c,0])
                # for index in range(Xtest.shape[1]):
                #     if(Xtest[d,index] == 1):
                #         prosteriorProb[d,c] += np.log10(self.condProb[c,index])
                #     else:
                #         prosteriorProb[d,c] += np.log10(1 - self.condProb[c,index])
                Z = Xtest[d,:]
                one_ind = np.where(Z==1)[0]
                # print(one_ind)
                # zero_ind = np.where(Z==0)[0]
                zero_ind = np.delete(np.arange(0,Xtest.shape[0]),one_ind,0)
                # print("Done With finding 0s")
                row_ones = np.ones((1,zero_ind.shape[0]))

                prosteriorProb[d,c] += np.sum(np.log10(self.condProb[c,one_ind]))
                prosteriorProb[d,c] +=  np.sum(np.log10(row_ones-self.condProb[c,zero_ind]))
                    # print(self.condProb[c, index])

            predLabel[d,0] = np.argmax(prosteriorProb[d,:])
            # print(prosteriorProb[d, :])
            # print(predLabel[d,0])
            # if d == 5:
            #     return
        print('Time taken for predict function::: {} ', {time.time()-start_time})
        return predLabel

SKLearn Classifiers: Decision Trees & Logistic Regression

In [23]:
# Function returns a desired SKLearn classifier at run time
# Note: classifierEnum can take two values 0 or 1 read code to know which classifier
# you would like to create at run time
def createSKLearnClassifier(classifierEnum):
    # Switching to create different sk learn classifiers at run time
    if classifierEnum == 0 :
        print(f"Created a DecisionTreeClassifier!\n")
        return DecisionTreeClassifier()
    elif classifierEnum == 1:
        print(f"Created a LogisticRegressionClassifier!\n")
        return LogisticRegression()
    elif classifierEnum == 2:
        print(f"Created a linear SVM classifier (supports high dim data and samples)!\n")
        return LinearSVC()
    elif classifierEnum == 3:
        print(f"Created a SVM classifier with SGD training!\n")
        return SGDClassifier()

K-Fold Cross Validation

In [24]:
# K-Fold Cross Validation function
def run_K_Fold_CrossValidation(X_words, Y, numFeatures, classifier, numFolds=None):

    # Convert X_words into a 1D numpy array for ease of indexing later
    X_words_numpy = np.array(X_words)

    """Starting K-Fold Cross Validation"""
    # Create a dictionary to hold our fold accuracies
    fold_Accuracy_Dict = {}

    # Create sklearn's K-Fold instance
    kf = KFold(n_splits=numFolds)

    # Find out how many splitting iterations
    print(f"Number of splitting iterations: {kf.get_n_splits(Y)}\n")
    # TO UPDATE: print(f"Number of splitting iterations: {kf.get_n_splits(Y)}\n")

    # Fold Iteration count
    foldCount = 0

    # Fold error sum tracker
    foldErrorSum = 0

    # Fold accuracy tracker
    foldAccuracySum = 0

    # K-Fold loop
    for training_indices, validation_indices in kf.split(Y):
    # TO UPDATE: for training_indices, validation_indices in kf.split(Y):

        print(f"Starting fold {foldCount + 1}.....")
        # print("Training: ", training_indices, "Validation: ", validation_indices)

        # Slicing our data to get current training and current validation sets
        curFoldTraining = X_words_numpy[training_indices]
        curFoldTrainingLabels = Y[training_indices,:]
        curFoldValidation = X_words_numpy[validation_indices]
        curFoldValidationLabels = Y[validation_indices,:]

        # print("Current fold training: \n", curFoldTraining) # if you want to see the sliced array for training
        # print(f"Current fold training shape: {curFoldTraining.shape}")
        # print("Current fold validation: \n", curFoldValidation) # if you want to see the sliced array for validation
        # print(f"Current fold validation shape: {curFoldValidation.shape}")

        # converting back into lists from numpy array
        curFoldTraining = list(curFoldTraining)
        curFoldValidation = list(curFoldValidation)
        # print(curFoldTraining)
        # print(f"Current training length as list: {len(curFoldTraining)}")
        # print(f"Current validation length as list: {len(curFoldValidation)}")

        # Converting into numpy arrays for the training and validation sets and also doing featureSelection
        Xtrain, Xval = featureSelection(curFoldTraining, curFoldValidation, numFeatures, 0)

        # Fit our model with training
        """TODO: use our naive bayes classifier's fit function to fit our model to our training set"""
        classifier.fit(Xtrain, curFoldTrainingLabels)
        # TO UPDATE: classifier.fit(Xtrain, curFoldTrainingLabels)

        # Predict the labels here
        """TODO: use our predict function to predict the labels on the validation set"""
        curFoldPredictedLabels = classifier.predict(Xval)

        # Calculate accuracy for this fold
        """TODO: find accuracy"""
        count = 0
        for i in range(0, curFoldPredictedLabels.shape[0]):
            if curFoldValidationLabels[i] == curFoldPredictedLabels[i]:
                count += 1
        currentFoldAccuracy = (count/curFoldPredictedLabels.shape[0])*100
        print(f"Accuracy for fold {foldCount + 1}: {currentFoldAccuracy}%\n")

        # Add the accuracy of this fold to the dictionary
        fold_Accuracy_Dict[str(foldCount + 1)] = float(currentFoldAccuracy)

        # Update fold error tracker & fold accuracy tracker
        foldErrorSum = foldErrorSum + (100.0 - currentFoldAccuracy)
        foldAccuracySum = foldAccuracySum + currentFoldAccuracy

        # Update fold number
        foldCount += 1

    # Graph the fold accuracies with the dictionary
    plt.figure(figsize=(7,7))
    plt.bar(fold_Accuracy_Dict.keys(), fold_Accuracy_Dict.values(), 0.3, color='b')
    plt.xlabel('Fold Number')
    plt.ylabel('Accuracy %')
    plt.title("K-Fold Accuracy Distribution of Current Model")
    plt.show()

    # Display this model's avg accuracy for the K-Fold
    avgAccuracyOfThisModel = float(float(foldAccuracySum)/float(foldCount))
    print(f"\nAvg accuracy for this model is: {avgAccuracyOfThisModel} %")

    # Display this model's avg error for the K-Fold
    avgErrorOfThisModel = float(float(foldErrorSum)/float(foldCount))
    print(f"Avg error for this model is: {avgErrorOfThisModel} %\n")

    # Returning the avg error, fold accuracy dictionary, and model accuracy
    return avgErrorOfThisModel, avgAccuracyOfThisModel, fold_Accuracy_Dict

In [25]:
# Creating naive bayes classifier
nb = Bernoulli_NB(8)

# Talk about feature selection function with alex tomorrow: prone to failure depending on how many features we choose
run_K_Fold_CrossValidation(X_words, Y, 1000, nb, numFolds=5)

Number of splitting iterations: 5

Starting fold 1.....
Length of ordered list: 13396
Starting the fit function::::
Time taken for Pior probs function::: 0.011912822723388672
Time taken for Condtional Prob::: 0.010971307754516602
Total time by fit function::: 0.022884130477905273
Starting the Predicting Function: tick tock... tick tock
Time taken for predict function::: {}  {0.20949625968933105}
Accuracy for fold 1: 70.5%

Starting fold 2.....
Length of ordered list: 13349


<ipython-input-22-db8a630cfebc>:118: DeprecationWarning: in the future out of bounds indices will raise an error instead of being ignored by `numpy.delete`.
  zero_ind = np.delete(np.arange(0,Xtest.shape[0]),one_ind,0)


KeyboardInterrupt: 

In [ ]:
# # Superclass for Bernouilli Naive Bayes Classifiers
# class Bernoulli_NB():
#     #Class constructor
#     def __int__(self,num_classes):
#       self.num_classes = num_classes
#       self.condProb = None
#       self.priorProb = None
#
#     ####################################################################################################
#     ####################################################################################################
#
#     # Calculate the probability of Y = 1.
#     # Returns a column vector for which the ith entry is P(Yi=1)
#     # and i refers to a class label.
#     def probY(self,Y):
#       num_labels = self.num_classes
#       prob = np.zeros((num_labels,1))  #Probability vector P(Yi=1)
#
#       for i in range(0,Y.shape[0]):
#         # Assumption: labels are integers ranging from 0 to num_labels - 1
#         for label in range(0, num_labels):
#           if Y[i,0] == label:
#             prob[label,0] += 1
#           elif Y[i,0] > (num_labels - 1):
#             print("Y at index " + str(i) + " is an invalid class label")
#             break
#       # Divide by the total # of labels input labels Y
#       prob = prob/Y.shape[0]
#       return prob
#
#     ####################################################################################################
#     ####################################################################################################
#     # Calculate the probability of X = 1 given a class label Yi (number).
#     # Returns a row vector for which the jth entry is P(Xj=1|Yi)
#     # and j refers to a feature.
#     # Note: not implemented with the bias term in mind.
#     def probXGivenYi(self, X, Y, label):
#       prob = np.zeros((1,X.shape[1]))  # Conditional probability vector P(Xj=1|Yi)
#       denominator = 0                  # Number of times label Yi appears in Y
#
#       for i in range(0, X.shape[0]):
#         if Y[i,0] == label:
#           denominator += 1
#           prob = prob + X[i,:]
#
#       # Laplace smoothing
#       prob = prob + np.ones((1,X.shape[1]))
#       denominator += 2
#
#       prob = prob/denominator
#       return prob
#     ####################################################################################################
#     ####################################################################################################
#     # Write the Bernoulli Naive  Bayes Method Here
#     def fit(self,Xtrain,Y):
#         self.condProb = np.zeros((self.num_classes,Xtrain.shape[1]))
#         self.priorProb = self.probY(Y, self.num_classes)
#
#         for c in range(self.num_classes):
#             self.condProb[c,:] = self.probXGivenYi(Xtrain, Y, c)
#
#     ####################################################################################################
#     ####################################################################################################
#
#     def predict(self,Xtest):
#
#         predLabel = np.zeros((Xtest.shape[0],1))
#         prosteriorProb = np.zeros((Xtest.shape[0],8))
#
#         for d in range(Xtest.shape[0]):
#             for c in range(8):
#                  for index in range(Xtest.shape[1]):
#                      prosteriorProb[d,c] = np.log10(self.priorProb[1,c])
#
#                      if(Xtest(d,index) ==1):
#                          prosteriorProb[d,c] += np.log10(self.condProb[c,index])
#                      else:
#                          prosteriorProb[d,c] += np.log10(1 - self.condProb[c,index])
#             predLabel[d,1] = np.argmax(prosteriorProb[d,:])
#         return predLabel


